# Imports

In [1]:
# Libraries
import pandas as pd
import time
import random

# Objects
from PI_RPS.Mechanics.Environment.environment import Environment
from PI_RPS.Mechanics.Actors.Carriers.cost_bidding_carrier import CostBiddingCarrier

# Tools
from PI_RPS.Games.init_tools import load_realistic_nodes_and_shippers_to_env

# Variables
from PI_RPS.Games.init_tools import nb_hours_per_time_unit, t_c_mu, t_c_sigma, ffh_c_mu, ffh_c_sigma

In [2]:
n_carriers_per_node = 15  # @param {type:"integer"}

shippers_reserve_price_per_distance = 1200.  # @param{type:"number"}
shipper_default_reserve_price = 20000.  # @param{type:"number"}
init_node_weights_distance_scaling_factor = 500.  # @param{type:"number"}
# not used if initialized by artificial weights
max_node_weights_distance_scaling_factor = 500. * 1.3  # @param{type:"number"}
# should be big enough to be unrealistic.
node_auction_cost = 0.  # @param{type:"number"}
node_nb_info = 100  # @param{type:"integer"}
max_nb_infos_per_load = 15  # @param{type:"integer"}

max_time_not_at_home = 30  # @param {type:"integer"}

In [3]:
def new_environment():
    e = Environment(nb_hours_per_time_unit=nb_hours_per_time_unit,
                    max_nb_infos_per_load=max_nb_infos_per_load,
                    init_node_weights_distance_scaling_factor=init_node_weights_distance_scaling_factor,
                    max_node_weights_distance_scaling_factor=max_node_weights_distance_scaling_factor)

    load_realistic_nodes_and_shippers_to_env(e=e,
                                             node_nb_info=node_nb_info,
                                             shippers_reserve_price_per_distance=shippers_reserve_price_per_distance,
                                             shipper_default_reserve_price=shipper_default_reserve_price,
                                             node_auction_cost=node_auction_cost,
                                             weights_file_name='weights_' + str(node_auction_cost) + '_' + str(n_carriers_per_node) + '.json')

    weight_master = e.nodes[0].weight_master

    counter = {}
    for k in range(n_carriers_per_node * len(e.nodes)):
        node = e.nodes[k % len(e.nodes)]
        if node in counter.keys():
            counter[node] += 1
        else:
            counter[node] = 1
        road_costs = random.normalvariate(mu=t_c_mu, sigma=t_c_sigma)
        drivers_costs = random.normalvariate(mu=ffh_c_mu, sigma=ffh_c_sigma)

        CostBiddingCarrier(name=node.name + '_' + str(counter[node]),
                           home=node,
                           in_transit=False,
                           next_node=node,
                           time_to_go=0,
                           load=None,
                           environment=e,
                           episode_expenses=[],
                           episode_revenues=[],
                           this_episode_expenses=[],
                           this_episode_revenues=0,
                           transit_cost=road_costs,
                           far_from_home_cost=drivers_costs,
                           time_not_at_home=0,
                           max_time_not_at_home=max_time_not_at_home,
                           t_c_mu=t_c_mu,
                           t_c_sigma=t_c_sigma,
                           ffh_c_mu=ffh_c_mu,
                           ffh_c_sigma=ffh_c_sigma,
                           too_high_bid=shipper_default_reserve_price)
    return e

# Data structure

# Loop

In [22]:
carriers_cols = ['run_id', 'name', 'home', 't_c', 'ffh_c', 'profit', 'nb_time_step']
loads_cols = ['run_id', 'load_id', 'departure', 'arrival', 'is_arrived', 'is_discarded', 'in_transit']
movements_cols = ['run_id', 'load_id', 'step', 'origin', 'destination', 'carrier', 'cost', 'reserve_price_involved']
carriers_df = pd.DataFrame(columns=carriers_cols)
loads_df = pd.DataFrame(columns=loads_cols)
movements_df = pd.DataFrame(columns=movements_cols)

In [23]:
def run(e, nb_iter_per_run, run_number):
    # Running environment
    for _ in range(nb_iter_per_run):
        e.iteration()

    # Getting data
    carriers_data = []
    for carrier_p in e.carriers:
        if len(carrier_p.episode_revenues) > 1:
            carriers_data.append([run_number, carrier_p.name,
                                  carrier_p.home.name,
                                  carrier_p.t_c,
                                  carrier_p.ffh_c,
                                  sum(carrier_p.episode_revenues[1:]) + sum(carrier_p.episode_expenses[1:]),
                                  nb_iter_per_run])
        else:
            carriers_data.append([run_number,
                                  carrier_p.name,
                                  carrier_p.home.name,
                                  carrier_p.t_c,
                                  carrier_p.ffh_c,
                                  0.,
                                  nb_iter_per_run])
            
    new_carriers_df = pd.DataFrame(columns=carriers_cols, data=carriers_data)
    
    loads_data = []
    movements_data = []
    loads = e.loads
    for load_p_k in range(len(loads)):
        load_p = loads[load_p_k]
        movements = load_p.movements
        loads_data.append([run_number,
                           load_p_k,
                           load_p.departure.name,
                           load_p.arrival.name,
                           load_p.is_arrived,
                           load_p.is_discarded,
                           load_p.in_transit
                          ])
        
        for step_k in range(len(movements)):
            movement = movements[step_k]
            movements_data.append([run_number,
                               load_p_k,
                               step_k,
                               movement[0].name,
                               movement[1].name,
                               movement[2].name,
                               movement[3] + movement[4],
                               movement[5]  # this sould be the involvment of reserve price in the movement
                              ])
            
    
    
    new_loads_df = pd.DataFrame(columns=loads_cols, data=loads_data)
    new_movements_df = pd.DataFrame(columns=movements_cols, data=movements_data)
    return new_carriers_df, new_loads_df, new_movements_df

In [24]:
nb_loop = 2
nb_iteration_per_run = 500
start_time = time.time()

for k in range(nb_loop):
    e = new_environment()
    new_carriers_df, new_loads_df, new_movements_df = run(e=e,
                                                          nb_iter_per_run=nb_iteration_per_run,
                                                          run_number=k)
    
    carriers_df = pd.concat(objs=[carriers_df, new_carriers_df], ignore_index=True)
    loads_df = pd.concat(objs=[loads_df, new_loads_df], ignore_index=True)
    movements_df = pd.concat(objs=[movements_df, new_movements_df], ignore_index=True)

end_time = time.time()
delta = int(end_time - start_time)
delta_h = delta // 3600
delta_m = (delta % 3600) // 60
delta_s = (delta % 3600) % 60
print("Total time:", "{}h{}m{}s".format(delta_h, delta_m, delta_s))

Total time: 0h0m8s
